In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8041056107924830651, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5126935124165143211
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 32
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)
epochs = 15

In [6]:
train_dir = 'data/train/'
test_dir = 'data/test'

In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.3,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 34909 images belonging to 6 classes.
Found 14958 images belonging to 6 classes.
Found 13578 images belonging to 6 classes.


In [8]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 34909
nb_validation_samples: 14958
nb_test_samples: 13578

predict_size_train: 1091
predict_size_validation: 468
predict_size_test: 425

 num_classes: 6


In [9]:
# os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "Xception_Dense201_InceptionV3_descriptors"

In [10]:
# vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
# vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
# resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
# nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
# nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
# mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"

In [11]:
# from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
# from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception, preprocess_input
from keras.applications import DenseNet201
# from keras.applications import DenseNet121
# from keras.applications import ResNet50
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from keras.applications import NASNetLarge, NASNetMobile
# from keras.applications import MobileNet

In [12]:
input_tensor = Input(shape = input_shape)  

base_model1=InceptionV3(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)
base_model2=Xception(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)
base_model3=DenseNet201(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)

x1 = base_model1.output
x1 = GlobalAveragePooling2D()(x1)

x2 = base_model2.output
x2 = GlobalAveragePooling2D()(x2)

x3 = base_model3.output
x3 = GlobalAveragePooling2D()(x3)

merge = concatenate([x1, x2, x3])
predictions = Dense(num_classes, activation='softmax')(merge)

model = Model(inputs=input_tensor,outputs=predictions)

In [13]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

0 input_1
1 zero_padding2d_1
2 conv1/conv
3 conv1/bn
4 conv1/relu
5 zero_padding2d_2
6 pool1
7 conv2_block1_0_bn
8 conv2_block1_0_relu
9 conv2_block1_1_conv
10 conv2_block1_1_bn
11 conv2_block1_1_relu
12 conv2_block1_2_conv
13 conv2_block1_concat
14 conv2_block2_0_bn
15 conv2_block2_0_relu
16 conv2_block2_1_conv
17 conv2_block2_1_bn
18 conv2_block2_1_relu
19 conv2_block2_2_conv
20 conv2_block2_concat
21 conv2_block3_0_bn
22 conv2_block3_0_relu
23 conv2_block3_1_conv
24 conv2_block3_1_bn
25 conv2_block3_1_relu
26 conv2_block3_2_conv
27 conv2_block3_concat
28 conv2_block4_0_bn
29 conv2_block4_0_relu
30 conv2_block4_1_conv
31 conv2_block4_1_bn
32 conv2_block4_1_relu
33 conv2_block4_2_conv
34 conv2_block4_concat
35 conv2_block5_0_bn
36 conv2_block5_0_relu
37 conv2_block5_1_conv
38 conv2_block5_1_bn
39 conv2_block5_1_relu
40 conv2_block5_2_conv
41 conv2_block5_concat
42 conv2_block6_0_bn
43 conv2_block6_0_relu
44 conv2_block6_1_conv
45 conv2_block6_1_bn
46 conv2_block6_1_relu
47 conv2_block

697 mixed2
698 block3_sepconv1_act
699 conv5_block17_1_relu
700 conv2d_28
701 block3_sepconv1
702 conv5_block17_2_conv
703 batch_normalization_28
704 block3_sepconv1_bn
705 conv5_block17_concat
706 activation_28
707 block3_sepconv2_act
708 conv5_block18_0_bn
709 conv2d_29
710 block3_sepconv2
711 conv5_block18_0_relu
712 batch_normalization_29
713 block3_sepconv2_bn
714 conv2d_96
715 conv5_block18_1_conv
716 activation_29
717 block3_pool
718 batch_normalization_96
719 conv5_block18_1_bn
720 conv2d_27
721 conv2d_30
722 add_2
723 conv5_block18_1_relu
724 batch_normalization_27
725 batch_normalization_30
726 block4_sepconv1_act
727 conv5_block18_2_conv
728 activation_27
729 activation_30
730 max_pooling2d_3
731 block4_sepconv1
732 conv5_block18_concat
733 mixed3
734 block4_sepconv1_bn
735 conv5_block19_0_bn
736 conv2d_35
737 block4_sepconv2_act
738 conv5_block19_0_relu
739 batch_normalization_35
740 block4_sepconv2
741 conv5_block19_1_conv
742 activation_35
743 block4_sepconv2_bn
744 conv2

In [15]:
c1 = model.layers[9].output 
c1 = GlobalAveragePooling2D()(c1)       

c2 = model.layers[55].output
c2 = GlobalAveragePooling2D()(c2)       

c3 = model.layers[143].output
c3 = GlobalAveragePooling2D()(c3)       

c4 = model.layers[483].output
c4 = GlobalAveragePooling2D()(c4) 

c5 = model.layers[571].output
c5 = GlobalAveragePooling2D()(c5) 

c6 = model.layers[585].output
c6 = GlobalAveragePooling2D()(c6) 

c7 = model.layers[601].output
c7 = GlobalAveragePooling2D()(c7) 

c8 = model.layers[634].output
c8 = GlobalAveragePooling2D()(c8) 

c9 = model.layers[677].output
c9 = GlobalAveragePooling2D()(c9) 

c10 = model.layers[700].output
c10 = GlobalAveragePooling2D()(c10) 

c11 = model.layers[720].output
c11 = GlobalAveragePooling2D()(c11) 

c12 = model.layers[756].output
c12 = GlobalAveragePooling2D()(c12) 

c13 = model.layers[781].output
c13 = GlobalAveragePooling2D()(c13) 

c14 = model.layers[845].output
c14 = GlobalAveragePooling2D()(c14) 

c15 = model.layers[909].output
c15 = GlobalAveragePooling2D()(c15) 

c16 = model.layers[973].output
c16 = GlobalAveragePooling2D()(c16) 

c17 = model.layers[1040].output
c17 = GlobalAveragePooling2D()(c17) 

c18 = model.layers[1068].output
c18 = GlobalAveragePooling2D()(c18) 

c19 = model.layers[1116].output
c19 = GlobalAveragePooling2D()(c19) 

c20 = model.layers[1123].output
c20 = GlobalAveragePooling2D()(c20) 

con = concatenate([c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15, c16, c17, c18, c19, c20])

bottleneck_final_model = Model(inputs=model.input, outputs=con)

In [16]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train, max_q_size=1, pickle_safe=False)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [17]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [18]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [19]:

import tensorflow as tf
from keras.backend.tensorflow_backend import get_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import set_session

def reset_keras_tf_session():
    """
    this function clears the gpu memory and set the 
    tf session to not use the whole gpu
    """
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     set_session(tf.Session(config=config))

reset_keras_tf_session()

In [20]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [21]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [22]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
  
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0)
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [23]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [24]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 300, trials=trials)
print(best)

Parameters testing:  {'lr': 0.01, 'beta_2': 0.5, 'beta_1': 0.4}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 4.4847 - acc: 0.4193 - val_loss: 1.3536 - val_acc: 0.4396
Epoch 2/15
 - 4s - loss: 1.6004 - acc: 0.5066 - val_loss: 1.4222 - val_acc: 0.4869
Epoch 3/15
 - 4s - loss: 1.6559 - acc: 0.5404 - val_loss: 5.5776 - val_acc: 0.4486
Epoch 4/15
 - 4s - loss: 1.6726 - acc: 0.5501 - val_loss: 3.3445 - val_acc: 0.4248
Epoch 5/15
 - 4s - loss: 1.6689 - acc: 0.5641 - val_loss: 1.9990 - val_acc: 0.4800
Epoch 00005: early stopping
14958/14958 [==============================] - 1s 38us/step
Validation Accuracy: 48.0011%
Validation Loss: 1.9990094679073875
Test Accuracy: 0.6140816025924289
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'lr': 0.01, 'beta_2': 0.5, 'beta_1': 0.2}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s -

Epoch 3/15
 - 4s - loss: 1.2771 - acc: 0.5298 - val_loss: 1.3621 - val_acc: 0.4394
Epoch 4/15
 - 4s - loss: 1.1523 - acc: 0.6069 - val_loss: 1.3007 - val_acc: 0.4620
Epoch 5/15
 - 4s - loss: 1.0697 - acc: 0.6403 - val_loss: 1.2408 - val_acc: 0.4850
Epoch 6/15
 - 4s - loss: 1.0048 - acc: 0.6659 - val_loss: 1.1937 - val_acc: 0.4992
Epoch 7/15
 - 4s - loss: 0.9570 - acc: 0.6818 - val_loss: 1.1452 - val_acc: 0.5214
Epoch 8/15
 - 4s - loss: 0.9068 - acc: 0.7002 - val_loss: 1.1066 - val_acc: 0.5370
Epoch 9/15
 - 4s - loss: 0.8715 - acc: 0.7114 - val_loss: 1.0764 - val_acc: 0.5498
Epoch 10/15
 - 4s - loss: 0.8428 - acc: 0.7182 - val_loss: 1.0455 - val_acc: 0.5612
Epoch 11/15
 - 4s - loss: 0.8113 - acc: 0.7305 - val_loss: 1.0267 - val_acc: 0.5673
Epoch 12/15
 - 4s - loss: 0.7913 - acc: 0.7343 - val_loss: 1.0021 - val_acc: 0.5784
Epoch 13/15
 - 4s - loss: 0.7692 - acc: 0.7411 - val_loss: 0.9865 - val_acc: 0.5858
Epoch 14/15
 - 4s - loss: 0.7501 - acc: 0.7466 - val_loss: 0.9676 - val_acc: 0.5948

 - 4s - loss: 0.3976 - acc: 0.8620 - val_loss: 0.6959 - val_acc: 0.7288
Epoch 13/15
 - 4s - loss: 0.3934 - acc: 0.8642 - val_loss: 0.7064 - val_acc: 0.7216
Epoch 14/15
 - 4s - loss: 0.3848 - acc: 0.8691 - val_loss: 0.6896 - val_acc: 0.7298
Epoch 00014: early stopping
14958/14958 [==============================] - 1s 41us/step
Validation Accuracy: 72.9777%
Validation Loss: 0.689608226000649
Test Accuracy: 0.8389306230667256
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'lr': 1e-05, 'beta_2': 0.5, 'beta_1': 0.9}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.0546 - acc: 0.6375 - val_loss: 0.9822 - val_acc: 0.5785
Epoch 2/15
 - 4s - loss: 0.6427 - acc: 0.7772 - val_loss: 0.8152 - val_acc: 0.6675
Epoch 3/15
 - 4s - loss: 0.5507 - acc: 0.8133 - val_loss: 0.7735 - val_acc: 0.6910
Epoch 4/15
 - 4s - loss: 0.5020 - acc: 0.8285 - val_loss: 

Epoch 2/15
 - 4s - loss: 1.5201 - acc: 0.4144 - val_loss: 1.4402 - val_acc: 0.3793
Epoch 3/15
 - 4s - loss: 1.2992 - acc: 0.5136 - val_loss: 1.3641 - val_acc: 0.4033
Epoch 4/15
 - 4s - loss: 1.1782 - acc: 0.5776 - val_loss: 1.3023 - val_acc: 0.4267
Epoch 5/15
 - 4s - loss: 1.0903 - acc: 0.6208 - val_loss: 1.2432 - val_acc: 0.4559
Epoch 6/15
 - 4s - loss: 1.0252 - acc: 0.6421 - val_loss: 1.2042 - val_acc: 0.4691
Epoch 7/15
 - 4s - loss: 0.9780 - acc: 0.6599 - val_loss: 1.1708 - val_acc: 0.4838
Epoch 8/15
 - 4s - loss: 0.9330 - acc: 0.6787 - val_loss: 1.1388 - val_acc: 0.4996
Epoch 9/15
 - 4s - loss: 0.9037 - acc: 0.6862 - val_loss: 1.1098 - val_acc: 0.5131
Epoch 10/15
 - 4s - loss: 0.8671 - acc: 0.6973 - val_loss: 1.0838 - val_acc: 0.5281
Epoch 11/15
 - 4s - loss: 0.8390 - acc: 0.7070 - val_loss: 1.0586 - val_acc: 0.5397
Epoch 12/15
 - 4s - loss: 0.8167 - acc: 0.7158 - val_loss: 1.0420 - val_acc: 0.5460
Epoch 13/15
 - 4s - loss: 0.7980 - acc: 0.7224 - val_loss: 1.0167 - val_acc: 0.5605


Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 6s - loss: 2.6970 - acc: 0.1395 - val_loss: 1.8547 - val_acc: 0.1786
Epoch 2/15
 - 4s - loss: 2.4227 - acc: 0.1694 - val_loss: 1.7285 - val_acc: 0.2151
Epoch 3/15
 - 4s - loss: 2.2383 - acc: 0.2014 - val_loss: 1.6647 - val_acc: 0.2280
Epoch 4/15
 - 4s - loss: 2.1112 - acc: 0.2244 - val_loss: 1.6319 - val_acc: 0.2300
Epoch 5/15
 - 4s - loss: 2.0001 - acc: 0.2451 - val_loss: 1.6111 - val_acc: 0.2512
Epoch 6/15
 - 4s - loss: 1.9228 - acc: 0.2667 - val_loss: 1.5941 - val_acc: 0.2750
Epoch 7/15
 - 4s - loss: 1.8583 - acc: 0.2790 - val_loss: 1.5775 - val_acc: 0.2974
Epoch 8/15
 - 4s - loss: 1.8032 - acc: 0.3000 - val_loss: 1.5620 - val_acc: 0.3162
Epoch 9/15
 - 4s - loss: 1.7383 - acc: 0.3175 - val_loss: 1.5476 - val_acc: 0.3279
Epoch 10/15
 - 4s - loss: 1.6787 - acc: 0.3388 - val_loss: 1.5324 - val_acc: 0.3410
Epoch 11/15
 - 4s - loss: 1.6404 - acc: 0.3494 - val_loss: 1.5195 - val_acc: 0.3498
Epoch 12/15
 - 4s - loss: 1.5881 - 

...

Epoch 8/15
 - 12s - loss: 0.4195 - acc: 0.8592 - val_loss: 0.7435 - val_acc: 0.7345
Epoch 9/15
 - 12s - loss: 0.4323 - acc: 0.8550 - val_loss: 0.8180 - val_acc: 0.7216
Epoch 10/15
 - 13s - loss: 0.4244 - acc: 0.8606 - val_loss: 0.7523 - val_acc: 0.7574
Epoch 11/15
 - 13s - loss: 0.4222 - acc: 0.8597 - val_loss: 0.7394 - val_acc: 0.7591
Epoch 12/15
 - 11s - loss: 0.4231 - acc: 0.8589 - val_loss: 0.9468 - val_acc: 0.7282
Epoch 13/15
 - 13s - loss: 0.4312 - acc: 0.8571 - val_loss: 0.8308 - val_acc: 0.7025
Epoch 14/15
 - 13s - loss: 0.4412 - acc: 0.8542 - val_loss: 0.8506 - val_acc: 0.7232
Epoch 00014: early stopping
14958/14958 [==============================] - 2s 156us/step
Validation Accuracy: 72.3158%
Validation Loss: 0.850555824915585
Test Accuracy: 0.8520400648107233
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'lr': 1e-05, 'beta_2': 0.6, 'beta_1': 0.8}
Tra

Epoch 4/15
 - 11s - loss: 0.3852 - acc: 0.8651 - val_loss: 0.7162 - val_acc: 0.7355
Epoch 5/15
 - 11s - loss: 0.3657 - acc: 0.8702 - val_loss: 0.7059 - val_acc: 0.7394
Epoch 6/15
 - 12s - loss: 0.3512 - acc: 0.8752 - val_loss: 0.6631 - val_acc: 0.7572
Epoch 7/15
 - 12s - loss: 0.3404 - acc: 0.8787 - val_loss: 0.6315 - val_acc: 0.7688
Epoch 8/15
 - 12s - loss: 0.3292 - acc: 0.8841 - val_loss: 0.6446 - val_acc: 0.7575
Epoch 9/15
 - 13s - loss: 0.3190 - acc: 0.8870 - val_loss: 0.6299 - val_acc: 0.7716
Epoch 10/15
 - 11s - loss: 0.3148 - acc: 0.8888 - val_loss: 0.7071 - val_acc: 0.7345
Epoch 11/15
 - 12s - loss: 0.3104 - acc: 0.8909 - val_loss: 0.6525 - val_acc: 0.7652
Epoch 12/15
 - 12s - loss: 0.3008 - acc: 0.8938 - val_loss: 0.6654 - val_acc: 0.7462
Epoch 00012: early stopping
14958/14958 [==============================] - 3s 173us/step
Validation Accuracy: 74.6156%
Validation Loss: 0.6653780131667418
Test Accuracy: 0.8689792311091471
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**

Epoch 2/15
 - 14s - loss: 0.4688 - acc: 0.8354 - val_loss: 0.6461 - val_acc: 0.7538
Epoch 3/15
 - 13s - loss: 0.4168 - acc: 0.8520 - val_loss: 0.6965 - val_acc: 0.7283
Epoch 4/15
 - 13s - loss: 0.3883 - acc: 0.8619 - val_loss: 0.6486 - val_acc: 0.7492
Epoch 5/15
 - 13s - loss: 0.3685 - acc: 0.8685 - val_loss: 0.6620 - val_acc: 0.7470
Epoch 00005: early stopping
14958/14958 [==============================] - 3s 174us/step
Validation Accuracy: 74.6958%
Validation Loss: 0.6620210173861963
Test Accuracy: 0.8564589777581382
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'lr': 0.0001, 'beta_2': 0.7, 'beta_1': 0.3}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 22s - loss: 0.6676 - acc: 0.7614 - val_loss: 0.7016 - val_acc: 0.7257
Epoch 2/15
 - 12s - loss: 0.4577 - acc: 0.8382 - val_loss: 0.7401 - val_acc: 0.7260
Epoch 3/15
 - 14s - loss: 0.4123 - acc: 

Parameters testing:  {'lr': 1e-07, 'beta_2': 0.995, 'beta_1': 0.9}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 22s - loss: 2.5457 - acc: 0.2086 - val_loss: 1.8921 - val_acc: 0.0869
Epoch 2/15
 - 11s - loss: 2.3519 - acc: 0.2225 - val_loss: 1.7933 - val_acc: 0.1048
Epoch 3/15
 - 12s - loss: 2.2025 - acc: 0.2372 - val_loss: 1.7380 - val_acc: 0.1602
Epoch 4/15
 - 12s - loss: 2.1010 - acc: 0.2525 - val_loss: 1.7046 - val_acc: 0.2189
Epoch 5/15
 - 12s - loss: 2.0009 - acc: 0.2721 - val_loss: 1.6802 - val_acc: 0.2486
Epoch 6/15
 - 14s - loss: 1.9268 - acc: 0.2878 - val_loss: 1.6601 - val_acc: 0.2680
Epoch 7/15
 - 13s - loss: 1.8569 - acc: 0.3075 - val_loss: 1.6411 - val_acc: 0.2821
Epoch 8/15
 - 13s - loss: 1.7875 - acc: 0.3242 - val_loss: 1.6237 - val_acc: 0.2941
Epoch 9/15
 - 13s - loss: 1.7271 - acc: 0.3404 - val_loss: 1.6062 - val_acc: 0.3069
Epoch 10/15
 - 13s - loss: 1.6803 - acc: 0.3538 - val_loss: 1.5903 - val_acc: 0.3182
Epoch 11/15
 - 11s - loss: 1.6249 - acc: 0

Epoch 7/15
 - 12s - loss: 0.3384 - acc: 0.8801 - val_loss: 0.6222 - val_acc: 0.7541
Epoch 8/15
 - 12s - loss: 0.3289 - acc: 0.8835 - val_loss: 0.6625 - val_acc: 0.7427
Epoch 00008: early stopping
14958/14958 [==============================] - 4s 237us/step
Validation Accuracy: 74.2680%
Validation Loss: 0.6625001117284992
Test Accuracy: 0.8555015466195316
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'lr': 0.001, 'beta_2': 0.7, 'beta_1': 0.1}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 23s - loss: 0.8285 - acc: 0.6979 - val_loss: 0.8570 - val_acc: 0.6999
Epoch 2/15
 - 11s - loss: 0.6768 - acc: 0.7683 - val_loss: 0.9235 - val_acc: 0.7116
Epoch 3/15
 - 11s - loss: 0.6717 - acc: 0.7808 - val_loss: 0.9196 - val_acc: 0.6770
Epoch 4/15
 - 11s - loss: 0.6712 - acc: 0.7853 - val_loss: 0.8348 - val_acc: 0.7353
Epoch 5/15
 - 11s - loss: 0.6776 - acc: 0